In [21]:
#importin the libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql import Window
import pandas as pd

In [2]:
#create spark session
spark = SparkSession \
        .builder \
        .appName("AirFlightData") \
        .getOrCreate()

In [116]:
pd.set_option('display.max_columns', None)

In [190]:
data_df= spark.read.csv('airline.csv.shuffle.bz2', header=True, inferSchema=True)


In [191]:
#EDA
#data_df.describe()

In [192]:
#data_df.printSchema()

In [193]:
#data_df.select([count(when(isnull(c) | isnan(c), c)).alias(c) for c in data_df.columns])

In [194]:
#ActualElapsedTime 
#data_df.summary()

In [195]:
data_df=data_df.na.drop(subset=["ArrDelay"])

In [196]:
data_df=data_df.na.fill(value=0)
data_df=data_df.na.fill('')

Best time of day/day of week/time of year to fly to
minimize delays

In [197]:
#Best time of the day to travel
#lets first find the highst deay time

GetHour = udf(lambda x: str(x)[0:-2] if str(x)[0:-2] else 0 , StringType())
data_hour_df=data_df.withColumn("Hour", GetHour(data_df['CRSDepTime']))

data_hour_df=data_hour_df.groupBy("Hour").agg(mean("ArrDelay").alias("average")).sort('average')


In [198]:
data_day_df=data_df.groupBy("DayOfWeek").agg(mean("ArrDelay").alias("average")).sort('average')


In [199]:
data_DayofMonth_df=data_df.groupBy("DayofMonth").agg(mean("ArrDelay").alias("average")).sort('average')


In [200]:
data_Month_df=data_df.groupBy("Month").agg(mean("ArrDelay").alias("average")).sort('average')


In [201]:
data_year_df=data_df.groupBy("Year").agg(mean("ArrDelay").alias("average")).sort('average')



In [202]:
data_FlightNum_df=data_df.groupBy("FlightNum").agg(mean("ArrDelay").alias("average")).sort('average')




In [203]:
data_Origin_DISt_df=data_df.groupBy("Origin","Dest").agg(mean("ArrDelay").alias("average")).sort('average')

In [204]:
#the Most company with delays
Most_delays_bycompany_df=data_df.groupBy("UniqueCarrier").agg(mean("ArrDelay").alias("average")).sort('average')


How does the number of people flying between different
locations change over time?

In [205]:
data_traveling_from_df=data_df.groupBy("Year","Origin").count().alias("count").sort('count')


In [206]:
data_traveling_To_df=data_df.groupBy("Year","Dest").count().alias("count").sort('count')


In [207]:
## main travel cancellation reasons
cancelled=data_df.groupBy('CancellationCode').count().alias("count").sort('count').where(data_df.CancellationCode !='')



How well does weather predict plane delays?

In [208]:
Bad_weather_days_df=data_df.where(data_df.CancellationCode =='B').select(data_df['Year'].alias('Y'),\
    data_df["Month"].alias('M'),data_df['DayofMonth'].alias('D'),'Origin','Dest').drop_duplicates()

Bad_weather_days_df=Bad_weather_days_df.join(data_df, (Bad_weather_days_df['Y'] == data_df['Year']) &\
    (Bad_weather_days_df['M'] == data_df['Month']) &\
       (Bad_weather_days_df['D'] == data_df['DayofMonth']), 'inner')



In [209]:
Bad_weather_days_dayofmonth_df=Bad_weather_days_df.groupBy("DayofMonth").agg(mean("ArrDelay").alias("average")).sort('average')



In [210]:
Bad_weather_month_df=Bad_weather_days_df.groupBy("Month").agg(mean("ArrDelay").alias("average")).sort('average')



In [211]:
Bad_weather_day_df=Bad_weather_days_df.groupBy("DayOfWeek").agg(mean("ArrDelay").alias("average")).sort('average')


In [212]:
data_hour_df.write.options(header=True).csv('delay by hour')
data_day_df.write.options(header=True).csv('delay by day')
data_DayofMonth_df.write.options(header=True).csv('delay by day of month')
data_Month_df.write.options(header=True).csv('delay by Month')
data_year_df.write.options(header=True).csv('delay by Year')
data_FlightNum_df.write.options(header=True).csv('delay by flight number')
data_Origin_DISt_df.write.options(header=True).csv('delay by origin and destnation')
Most_delays_bycompany_df.write.options(header=True).csv('delay by company')
data_traveling_from_df.write.options(header=True).csv('most airports that were origins by year')
data_traveling_To_df.write.options(header=True).csv('most airports that were Destnation by year')
cancelled.write.options(header=True).csv('most reasons the flights canceled for')

Bad_weather_days_dayofmonth_df.write.options(header=True).csv('Bad wether effect on delay by day of month')
Bad_weather_month_df.write.options(header=True).csv('Bad wether effect on delay by month')
Bad_weather_day_df.write.options(header=True).csv('Bad wether effect on delay by Day of the week')

Py4JJavaError: An error occurred while calling o4686.csv.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.jobAbortedError(QueryExecutionErrors.scala:651)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:278)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:186)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:113)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:111)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.executeCollect(commands.scala:125)
	at org.apache.spark.sql.execution.QueryExecution$$anonfun$eagerlyExecuteCommands$1.$anonfun$applyOrElse$1(QueryExecution.scala:98)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withNewExecutionId$6(SQLExecution.scala:109)
	

In [ ]:
import bz2

bfb = bz2.BZ2File('airline.csv', mode='wb')
bfb.co


PermissionError: [Errno 13] Permission denied: 'airline.csv'